# **Staff Planning for Insurance Company**
#### **By Shivndra Anupam, Sangeeta Das & Sukhada Karale**
An insurance company needs help in finding the optimal number of staff they need for the insurance application approval process. In the industry, the Full-Time Equivalent(FTE) of the staff is considered a continuous variable.

This company operates in three states: A, B, and C whose data has been provided in the file Staffing+Data.xlsx

The objective is to optimize the total cost of the application approval process with the conditions mentioned below:


*   The company can either handle an application with the staff that they hire or outsource it to a vendor. Assume that there is no capacity limitation to outsourcing.

*   If they hire staff, he/she can handle 40 insurance applications per month when he/she works 100% of the workdays. However, there are days that he/she will be unavailable to process applications due to training, off days, etc.


*   States A and B have a regulatory restriction that the outsourced insurance applications cannot be more than 30% and 40% of the total number of applications for each month, respectively.






In [ ]:
# Importing Libraries 
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc
!apt-get install -y -qq glpk-utils
import pandas as pd
import numpy as np
import math
from pyomo.environ import *

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from __future__ import division
from pyomo.opt import SolverFactory

In [ ]:
# Reading the data from Excel workbook
path = '/content/gdrive/MyDrive/Colab Notebooks/Staffing+Data.xlsx'

#Dataframe of the Demand for company in all states
State_Demand = pd.read_excel(path, sheet_name = 'DemandData')
#Dataframe of Staff Availability
Staff_Availability = pd.read_excel(path, sheet_name = 'StaffAvailability')
#Dataframe of Cost of staffing
Cost = pd.read_excel(path, sheet_name = 'Cost')
#Dataframe of Service Rate
Service_Rate = pd.read_excel(path, sheet_name = 'ServiceRate')

## Data pre-processing 

In [ ]:
# Create the required Python data structures for indexes and parameters
# Create a Dictionary object Demand with index: 'State' and 'Month' and Value as "Demand" column from the State_Demand Dataframe
Demand = State_Demand.set_index(['State','Month'])['Demand'].to_dict()
# Create a Dictionary object FTE_Sallary with index: 'State' and 'Month' and Value as "MonthlySalary" column from the State_Demand Dataframe
FTE_Sallary = Cost.set_index(['State','Month'])['MonthlySalary'].to_dict()
# Create a Dictionary object OutSourceCost with index: 'State' and 'Month' and Value as "UnitOutSourceCost" column from the cost Dataframe
UnitOutSourceCost = Cost.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()
#Defining service rate per month
FTE_AppServRate = 40

#Defining Staff Aviaibility with indexes
StaffAvPer = Staff_Availability.set_index(['State','Month'])['StaffAvPer'].to_dict()
StaffAv_LB = Staff_Availability.set_index(['State','Month'])['LB'].to_dict()
StaffAv_UB = Staff_Availability.set_index(['State','Month'])['UB'].to_dict()

#Getting unique values
State = Cost['State'].unique()
Month = Cost['Month'].unique()

# Question 1

The company wants to know the optimised staffing recommendations for the business case described. 
Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).



In [ ]:
#Answered in the report

**Note:** You can write the mathematical formulation directly in your report.

# Question 2

Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step. 

#### Expected output:

Create a data frame containing the number of outsourced applications  and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. 

In [ ]:
# Creating a model instance
model = ConcreteModel()

In [ ]:
# Define Pyomo sets and Parameters
model.S = Set(initialize=State.tolist(),doc='States')
model.M = Set(initialize=Month.tolist(),doc='Months')
model.demand = Param(model.S,model.M,initialize = Demand,doc='Demand')
model.sa = Param(model.S,model.M,initialize = StaffAvPer,doc = 'StaffAvPercent')

Decision variables

FTE-S,M where S∈State M∈Month 

Outsourcing-S,M where S∈State M∈Month 

In [ ]:
# Decision variables
model.F = Var(model.S,model.M,doc='No. of FTE',domain = NonNegativeReals)
model.O = Var(model.S,model.M,doc='No of Outsource App',domain = NonNegativeIntegers)

Constraints:

total_demandS,M ==  FTES,M∗40∗S and month of outsourcing

Outsourcing-S,M ≤ total_demand-S,M∗0.30 for state A

Outsourcing-S,M ≤ total_demand-S,M∗0.40 for state B

In [ ]:
# Constraints
#1. Total demand constraint:
def total_demand(m,S,M):
    return (m.F[S,M]*40*m.sa[S,M] + m.O[S,M] == m.demand[S,M])
            
model.total_demand = Constraint(model.S,model.M, rule=total_demand)

#2. Outsourcing and Demand constraint:
model.outsource_demand = ConstraintList()
for M in model.M:
    for S in model.S:
        if S == 'A':
            model.outsource_demand.add(expr = model.O[S,M] <= 0.3*model.demand[S,M])
        elif S == 'B':
            model.outsource_demand.add(expr = model.O[S,M] <= 0.4*model.demand[S,M])


Objective Function

min∑(FTE-S,M ∗ FTE_Salary-S,M   +   Outsourcing-S,M ∗ Outsourcing_Salary-S,M)

In [ ]:
# Objective function
model.value = Objective(expr = sum(sum(model.F[S,M]*FTE_Sallary[S,M] + model.O[S,M]*UnitOutSourceCost[S,M] for S in model.S) for M in model.M), sense= minimize)

In [ ]:
# Invoking the solver
result = SolverFactory('glpk').solve(model, tee=True)
result.write()

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmp3oqdqt0l.glpk.raw --wglp /tmp/tmpn1hq2drg.glpk.glp --cpxlp
 /tmp/tmpmyqi33h9.pyomo.lp
Reading problem data from '/tmp/tmpmyqi33h9.pyomo.lp'...
61 rows, 73 columns, 97 non-zeros
36 integer variables, none of which are binary
469 lines were read
Writing problem data to '/tmp/tmpn1hq2drg.glpk.glp'...
440 lines were written
GLPK Integer Optimizer, v4.65
61 rows, 73 columns, 97 non-zeros
36 integer variables, none of which are binary
Preprocessing...
Objective value =   1.796233645e+07
INTEGER OPTIMAL SOLUTION FOUND BY MIP PREPROCESSOR
Time used:   0.0 secs
Memory used: 0.1 Mb (72516 bytes)
Writing MIP solution to '/tmp/tmp3oqdqt0l.glpk.raw'...
143 lines were written
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# -------------------------------------------------

In [ ]:
# Print the value of the objective function
optimized_cost = model.value()
print(optimized_cost)
print("\n\n Seems like the company has to spend around {}m$ in total for the application approval process." .format((round(optimized_cost)/1000000,2)))

17962336.44859711


 Seems like the company has to spend around (17.962336, 2)m$ in total for the application approval process.


**`Checkpoint 1:`** Seems like the company has to spend around 17.9 m$ in total for the application approval process.

In [ ]:
# Creating dataframe for the results
#Creating an empty list
Output_optimised = []


for S in model.S:
    for M in model.M:
        
        #cost for the application approval process
        no_of_fte=model.F[S,M].value
        demand = model.demand[S,M]
        no_of_app_processed_by_outsc = model.O[S,M].value
        no_of_FTE_processed_app = model.F[S,M].value*40*model.sa[S,M]
        cost = model.F[S,M].value*FTE_Sallary[S,M] + model.O[S,M].value*UnitOutSourceCost[S,M]
        percent_outsourced_app = round((no_of_app_processed_by_outsc/demand)*100,2)
        avg_cost_per_app = round(cost/demand,2)
        
        Output_optimised.append([S,M, no_of_fte,demand, no_of_app_processed_by_outsc,no_of_FTE_processed_app, cost, percent_outsourced_app,avg_cost_per_app])
        
Output_optimised = pd.DataFrame(Output_optimised, columns = ['State', 'Month', 'No. of FTE','Demand','Applications processed by Outsource','Applications processed by FTE' , 'Estimated Cost', 'Percent outsourced Application', 'average cost per application'])

Output_optimised[['Estimated Cost in mn $']] = Output_optimised[['Estimated Cost']]/1000000
Output_optimised['Demand'] = Output_optimised['Demand'].astype(int)
Output_optimised['Applications processed by Outsource'] = Output_optimised['Applications processed by Outsource'].astype(int)
Output_optimised['Applications processed by FTE'] = Output_optimised['Applications processed by FTE'].astype(int)
Output_optimised = Output_optimised.round({"Estimated Cost":2, "No. of FTE":0, "No. of outsource application":0, "Estimated Cost in mn $":2})

In [ ]:
Output_optimised.head(10)

,State,Month,No. of FTE,Demand,Applications processed by Outsource,Applications processed by FTE,Estimated Cost,Percent outsourced Application,average cost per application,Estimated Cost in mn $
0,A,Jan,162.0,5240,0,5239,808641.98,0.0,154.32,0.81
1,A,Feb,160.0,4878,0,4877,802302.63,0.0,164.47,0.80
2,A,Mar,198.0,5942,0,5942,990333.33,0.0,166.67,0.99
3,A,Apr,72.0,2297,0,2297,358906.25,0.0,156.25,0.36
4,A,May,64.0,1992,0,1991,319230.77,0.0,160.26,0.32
5,A,Jun,78.0,2275,0,2275,389554.79,0.0,171.23,0.39
6,A,Jul,137.0,5334,1600,3734,974397.06,30.0,182.68,0.97
7,A,Aug,111.0,3371,0,3371,554440.79,0.0,164.47,0.55
8,A,Sep,116.0,3759,0,3759,580092.59,0.0,154.32,0.58
9,A,Oct,121.0,3529,0,3529,604280.82,0.0,171.23,0.60


In [ ]:
# Writing the results in to an Excel sheet
path1 = '/content/gdrive/MyDrive/Colab Notebooks/Output.xlsx'
Output_optimised.to_excel(path1, sheet_name = 'Optimised Sheet')

# Question 3

#### Worst-case and best-case analysis based on the staffs' availability.

Assuming that the distribution is the same across all the states,

#### 3.1 Worst case analysis 

- 3.1.1 What is the optimal number of staff members for the worst case? 

- 3.1.2 What is the percentage of outsourcing for the worst case? 

- 3.1.3 What is the average cost per application for the worst case?


#### 3.2 Best case analysis 

- 3.2.1 What is the optimal number of staff members for the best case? 

- 3.2.2 What is the percentage of outsourcing for the best case? 

- 3.2.3 What is the average cost per application for the best case?


#### Expected output:

For each of the subtasks (3.1 and 3.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application. 

### 3.1 Worst Case Analysis 

#### 3.1.1 Optimal number of staff members

In [ ]:
# Creating a model instance
model_worst = ConcreteModel()

In [ ]:
# Define Pyomo sets and Parameters
model_worst.S = Set(initialize=State.tolist(),doc='States')
model_worst.M = Set(initialize=Month.tolist(),doc='Months')
model_worst.demand = Param(model_worst.S,model_worst.M,initialize = Demand,doc='Demand')
model_worst.sa = Param(model_worst.S,model_worst.M,initialize = StaffAv_LB,doc = 'StaffLowerBoundPercent')

In [ ]:
# Decision variables
model_worst.F = Var(model_worst.S,model_worst.M,doc='No. of FTE',domain = NonNegativeReals)
model_worst.O = Var(model_worst.S,model_worst.M,doc='No of Outsource App',domain = NonNegativeIntegers)

In [ ]:
# Constraints
# 1. Demand Constraint
def total_demand(m, S, M):
    return (m.F[S,M] * 40 * m.sa[S,M] + m.O[S,M] == m.demand[S,M])


model_worst.total_demand = Constraint(model_worst.S, model_worst.M, rule=total_demand)

# . Outsource Demand Constraints
model_worst.outsource_demand = ConstraintList()
for M in model_worst.M:
    for S in model_worst.S:
        if S == 'A':
            model_worst.outsource_demand.add(expr = model_worst.O[S,M] <= 0.3*model_worst.demand[S,M])
        elif S == 'B':
            model_worst.outsource_demand.add(expr = model_worst.O[S,M] <= 0.4*model_worst.demand[S,M])

In [ ]:
# Objective function
model_worst.value = Objective(expr = sum(sum(model_worst.F[S,M]*FTE_Sallary[S,M]+ model_worst.O[S,M]*UnitOutSourceCost[S,M] for S in model_worst.S) for M in model_worst.M),sense= minimize)

In [ ]:
# Invoking the solver
result = SolverFactory('glpk').solve(model_worst)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 19599482.5165589
  Upper bound: 19599482.5165589
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.015285968780517578
# ----------------------------------------------------------
#   Solution Information
# -----------

In [ ]:
# Print the value of the objective function
optimized_worst_cost = model_worst.value()
print(optimized_worst_cost)
print("\n\n Seems like the company has to spend around {}m$ in total for the application approval process if the staffs are working with the minimum availability.." .format(round((optimized_worst_cost)/1000000,2)))

19599482.516558886


 Seems like the company has to spend around 19.6m$ in total for the application approval process if the staffs are working with the minimum availability..


**`Checkpoint 2:`** The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability.

In [ ]:
# Creating dataframe for the results
Output_worst = []

for S in model_worst.S:
    for M in model_worst.M:
       
        no_of_fte = model_worst.F[S,M].value
        demand = model_worst.demand[S,M]
        no_of_outsource_app = model_worst.O[S,M].value
        no_of_FTE_processed_app = model_worst.F[S,M].value*40*model_worst.sa[S,M]
        cost = model_worst.F[S,M].value*FTE_Sallary[S,M] + model_worst.O[S,M].value*UnitOutSourceCost[S,M]

        percent_outsourced_app = round((no_of_outsource_app/demand)*100,1)
        avg_cost_per_app = round(cost/demand,1)

        Output_worst.append([S,M, no_of_fte,demand, no_of_outsource_app,no_of_FTE_processed_app, cost, percent_outsourced_app,avg_cost_per_app])
       
    
Output_worst = pd.DataFrame(Output_worst, columns = ['State', 'Month', 'No. of FTE worst','Demand','Applications processed by Outsource-worst',"Applications processed by FTE-worst" , 'Estimated Cost worst', 'Percent outsourced Application worst', 'average cost per application worst'])

Output_worst['Estimated Cost in mn $ worst'] = Output_worst[['Estimated Cost worst']]/1000000
Output_worst['Demand'] = Output_worst['Demand'].astype(int)
Output_worst['Applications processed by Outsource-worst'] = Output_worst['Applications processed by Outsource-worst'].astype(int)
Output_worst['Applications processed by FTE-worst'] = Output_worst['Applications processed by FTE-worst'].astype(int)
c = Output_worst.round({"No. of FTE worst":1, "Applications processed by Outsource-worst":0, "Estimated Cost in mn $ worst":1})
#Output_worst[['Cost per application worst']] = Output_worst['Estimated Cost worst']/Output_worst['Demand']

total_outsourcing =  sum(Output_worst['Applications processed by Outsource-worst'])
total_cost = sum(Output_worst['Estimated Cost worst'])
optimal_staff_members = round(Output_worst['No. of FTE worst'].mean(),0)

total_demand =  sum(Output_worst['Demand'])
average_cost_per_appln = round(total_cost/total_demand,2)
percent_outsourced_app_total = round((total_outsourcing/total_demand)*100,1)
print("Total Outsourced Application: ", total_outsourcing)
print("Total Demand Application: ", total_demand)
print("Optimal Staff Members : ", optimal_staff_members)

Total Outsourced Application:  39808
Total Demand Application:  113294
Optimal Staff Members :  77.0


#### 3.1.2 Percentage of outsourced applications 

In [ ]:
# write your code here
print("Total Outsourced Application % is : ", percent_outsourced_app_total)

Total Outsourced Application % is :  35.1


#### 3.1.3 Average cost per application

In [ ]:
# write your code here
print("Average Cost per Application: ", average_cost_per_appln)

Average Cost per Application:  173.0


### 3.2  Best Case Analysis 

#### 3.2.1 Optimal number of staff members


In [ ]:
# Creating a model instance
modelB = ConcreteModel()

In [ ]:
# Define Pyomo sets and Parameters
modelB.S = Set(initialize=State.tolist(),doc='States')
modelB.M = Set(initialize=Month.tolist(),doc='Months')
modelB.demand = Param(modelB.S,modelB.M,initialize = Demand,doc='Demand')
modelB.sa = Param(modelB.S,modelB.M,initialize = StaffAv_UB,doc = 'StaffUpperBoundPercent')

In [ ]:
# Decision variables
modelB.F = Var(modelB.S,modelB.M,doc='No. of FTE',domain = NonNegativeReals)
modelB.O = Var(modelB.S,modelB.M,doc='No of Outsource App',domain = NonNegativeIntegers)

In [ ]:
# Constraints
# 1. Demand Constraint

def total_demand(m, S, M):
    return (m.F[S, M] * 40 * m.sa[S, M] + m.O[S, M] == m.demand[S, M])


modelB.total_demand = Constraint(modelB.S, modelB.M, rule=total_demand)

# 2. Outsource Demand Constraints
modelB.outsource_demand = ConstraintList()
for M in modelB.M:
    for S in modelB.S:
        if S == 'A':
            modelB.outsource_demand.add(expr = modelB.O[S,M] <= 0.3*modelB.demand[S,M])
        elif S == 'B':
            modelB.outsource_demand.add(expr = modelB.O[S,M] <= 0.4*modelB.demand[S,M])

In [ ]:
# Objective function
modelB.value = Objective(expr = sum(sum(modelB.F[S,M]*FTE_Sallary[S,M]+ modelB.O[S,M]*UnitOutSourceCost[S,M] for S in modelB.S) for M in modelB.M),sense= minimize)

In [ ]:
# Invoking the solver
result = SolverFactory('glpk').solve(modelB)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16527535.6377772
  Upper bound: 16527535.6377772
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.018615007400512695
# ----------------------------------------------------------
#   Solution Information
# -----------

In [ ]:
# Print the value of the objective function
# Print the value of the objective function
optimized_best_cost = modelB.value()
print(optimized_best_cost)
print("\n\n Seems like the company has to spend around {}m$ in total for the application approval process if the staffs are working with the maximum availability." .format(round((optimized_best_cost)/1000000,2)))

16527535.637777213


 Seems like the company has to spend around 16.53m$ in total for the application approval process if the staffs are working with the maximum availability.


**`Checkpoint 3:`** The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.

In [ ]:
# Creating dataframe for the results
Output_best = []

for S in modelB.S:
    for M in modelB.M:
        # cost for the application approval process
        no_of_fte = modelB.F[S,M].value
        demand = modelB.demand[S,M]
        no_of_outsource_app = modelB.O[S,M].value
        no_of_FTE_processed_app = modelB.F[S,M].value*40*modelB.sa[S,M]
        cost = modelB.F[S,M].value*FTE_Sallary[S,M] + modelB.O[S,M].value*UnitOutSourceCost[S,M]

        percent_outsourced_app = round((no_of_outsource_app/demand)*100,1)
        cost_per_app = round(cost/demand,1)

        Output_best.append([S,M, no_of_fte,demand, no_of_outsource_app,no_of_FTE_processed_app, cost, percent_outsourced_app,cost_per_app])

Output_best = pd.DataFrame(Output_best, columns = ['State', 'Month', 'No. of FTE best','Demand','Applications processed by Outsource-best',"Applications processed by FTE-best" , 'Estimated Cost best', 'Percent outsourced Application best',  'cost per application best'])
Output_best[['Estimated Cost in mn $ best']] = Output_best[['Estimated Cost best']]/1000000
Output_best['Demand'] = Output_best['Demand'].astype(int)
Output_best['Applications processed by Outsource-best'] = Output_best['Applications processed by Outsource-best'].astype(int)
Output_best['Applications processed by FTE-best'] = Output_best['Applications processed by FTE-best'].astype(int)
Output_best = Output_best.round({"No. of FTE best":1, "No. of outsource application best":0, "Estimated Cost in mn $ best":1})
#Output_best[['Cost per application best']] = Output_best['Estimated Cost best']/Output_best['Demand']

total_outsourcing =  sum(Output_best['Applications processed by Outsource-best'])
total_cost = sum(Output_best['Estimated Cost best'])
optimal_staff_members = round(Output_best['No. of FTE best'].mean(),0)

total_demand =  sum(Output_best['Demand'])
average_cost_per_appln = round(total_cost/total_demand,2)
percent_outsourced_app_total = round((total_outsourcing/total_demand)*100,1)
print("Total Outsourced Application: ", total_outsourcing)
print("Total Demand Application: ", total_demand)
print("Optimal Staff Members : ", optimal_staff_members)

Total Outsourced Application:  4652
Total Demand Application:  113294
Optimal Staff Members :  93.0


#### 3.2.2 Percentage of outsourced applications

In [ ]:
# write your code here
print("Total Outsourced Application % is : ", percent_outsourced_app_total)

Total Outsourced Application % is :  4.1


#### 3.2.3 Average cost per application

In [ ]:
# write your code here
print("Average Cost per Application: ", average_cost_per_appln)

Average Cost per Application:  145.88


# Question 4

#### Creating Visualisations

Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%). 
Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis.
Hint: Use the cost per application that you calculate in Questions 2 and 3 (i.e., the best case, and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.

In [ ]:
#creating excel files
path1 = '/content/gdrive/MyDrive/Colab Notebooks/Output.xlsx'
Output_optimised.to_excel(path1, sheet_name = 'Optimised Sheet')
Output_worst.to_excel(path1, sheet_name = 'Worst case')
Output_best.to_excel(path1, sheet_name = 'Best case')